# middleschool-cardlist

## Prepare the data

Download raw data from [MTGJSON](https://mtgjson.com/) (uncomment and run only once)


In [1]:
# !cd data
# !wget "https://mtgjson.com/api/v5/AllPrintings.json.bz2"
# !bunzip2 AllPrintings.json.bz2
# !cd -


The Raw data is very large, so let's make JSON files for all relevant sets

Note: this cell can take a couple minutes to run


In [2]:
setlist = ['4ED', 'ICE', 'CHR', 'HML', 'ALL', 'MIR', 'VIS', '5ED',
           'WTH', 'POR', 'TMP', 'STH', 'EXO', 'P02', 'USG', 'ULG',
           '6ED', 'UDS', 'PTK', 'S99', 'MMQ', 'NEM', 'PCY', 'S00',
           'INV', 'PLS', '7ED', 'APC', 'ODY', 'TOR', 'JUD', 'ONS',
           'LGN', 'SCG', 'PDRC', 'PHPR']
for set in setlist:
    command = 'cat data/AllPrintings.json | jq \'.data.\"' + \
        set + '\".cards\' > data/set_' + set + '.json'
    !{command}


Concatenate all set files into `middleschool.json`


In [9]:
command = "jq -s add data/set_* > data/middleschool.json"
!{command}


Create a list with each card's oracle ID, English name, and Japanese name


In [12]:
import json
import pandas as pd

with open("data/middleschool.json") as json_data:
    cards = json.loads(json_data.read())

column_names = ['oracle_id', 'name', 'name_ja']
middleschool_df = pd.DataFrame(columns=column_names)
for card in cards:
    oracle_id = card['identifiers']['scryfallOracleId']
    name = card['name']
    lang_ja = [lang for lang in card['foreignData']
               if lang['language'] == 'Japanese']
    if (len(lang_ja) > 0):
        name_ja = lang_ja[0]['name']
    else:
        name_ja = None
    temporary_df = pd.DataFrame({
        'oracle_id': [oracle_id],
        'name':      [name],
        'name_ja':   [name_ja]
    })
    middleschool_df = pd.concat([middleschool_df, temporary_df])

middleschool_df = middleschool_df.drop_duplicates(subset=['oracle_id'])
middleschool_df = middleschool_df.sort_values(by='name')
print(middleschool_df.shape[0], 'cards found')
print(middleschool_df.head())
print(middleschool_df.tail())


5793 cards found
                              oracle_id               name    name_ja
0  8adbba6e-03ef-4278-aec5-8a4496b377a8       Abandon Hope         断念
0  5a70ccfa-d12d-4e62-a1a4-f05cda2fd442  Abandoned Outpost  見捨てられた前哨地
0  c208b959-d0e4-4a9a-8255-2c7cc7596767    Abbey Gargoyles  修道院のガーゴイル
0  62e3f285-886c-414e-b4ff-403a7c01c23a       Abbey Matron       None
0  d0e1904e-1a37-41f6-8582-b9ea794bb886          Abduction         誘拐
                              oracle_id                      name    name_ja
0  ae8773a3-05f2-4074-9a53-033b0c127235  Zuo Ci, the Mocking Sage  嘲笑する仙人 左慈
0  c6eaa147-3566-43a9-999a-d58b877496f5            Zur's Weirding   ズアーの運命支配
0  ee0f883f-d7c9-4acf-a78f-f733b6f268d3           Zuran Enchanter       None
0  08cb8a30-9cb4-4517-bee5-8848aa60d1a2                 Zuran Orb       None
0  bc7b90b1-3517-4e5d-9bd8-68b4d8a259fd         Zuran Spellcaster       None


Exclude all cards banned in Middle School


In [13]:
banlist = ["Amulet of Quoz",
           "Balance",
           "Brainstorm",
           "Bronze Tablet",
           "Channel",
           "Dark Ritual",
           "Demonic Consultation",
           "Flash",
           "Goblin Recruiter",
           "Imperial Seal",
           "Jeweled Bird",
           "Lion's Eye Diamond",
           "Mana Crypt",
           "Mana Vault",
           "Memory Jar",
           "Mind's Desire",
           "Mind Twist",
           "Rebirth",
           "Strip Mine",
           "Tempest Efreet",
           "Timmerian Fiends",
           "Tolarian Academy",
           "Vampiric Tutor",
           "Windfall",
           "Yawgmoth's Bargain",
           "Yawgmoth's Will"]
print('Cards legal by set:', middleschool_df.shape[0])
banned_df = middleschool_df[pd.DataFrame(
    middleschool_df.name.tolist()).isin(banlist).any(1).values]
print('Banned cards:', banned_df.shape[0])
middleschool_df = pd.concat(
    [middleschool_df, banned_df]).drop_duplicates(keep=False)
print('Cards legal by set and not banned:', middleschool_df.shape[0])
middleschool_df = middleschool_df.reset_index(drop=True)
middleschool_df = middleschool_df[['oracle_id', 'name', 'name_ja']]


Cards legal by set: 5793
Banned cards: 26
Cards legal by set and not banned: 5767


Save the list to a CSV file and a JSON file


In [14]:
middleschool_df.to_csv('output/middleschool.csv')
middleschool_df.to_json('output/middleschool.json')
